Project: RAG using Ollama & ChromaDB <br>
Filename: rag.ipynb <br>
Title: RAG using gemma <br>
Author: Raghava | GitHub: @raghavtwenty  <br>
Date Created: August 29, 2024 | Last Updated: August 30, 2024  <br>
Language: Python | Version: 3.10.14 <br>

Install open-source embedding from ollama

In [1]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


Required imports

In [2]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import VectorDBQA
from langchain_community.chat_models import ChatOllama

File path

In [3]:
file_path = "diat.pdf"

Check for file

In [4]:
if file_path:
    # load the pdf file
    loader = PDFPlumberLoader(file_path=file_path)
    data = loader.load()
else:
    print("File doesn't exist")

View pdf

In [5]:
print(data[0].page_content)

Defence Institute of Advanced Technology
Girinagar, Pune 411025
School of Computer Engineering and Mathematical Sciences SoCE&MS
( )
ABOUT SoCE&MS
In the vibrant landscape of technological digital advancement, the School of Computer
Engineering and Mathematical Sciences (SoCE&MS) emerged in 2022 as a beacon of innovation
and progress. Born with the fusion of the Department of Computer Science and Engineering
(CSE), the Department of Applied Mathematics (AM) and Data Centre, the mission is to pioneer
breakthroughs in the realm of Computing and Mathematical Sciences. The CSE, our trailblazing
department was founded in 1987, with the first MTech course offered in 2009; while AM has been
integral part of our institute’s history since its inception. We stand on the shoulders of this rich
legacy as to strengthen and apply modern digitization, as a step into a future defined by
innovation, critical thinking, and scientific excellence.
As SoCE&MS, we offer a tapestry of FIVE Post-Graduation ac

Split and chuck it

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100)

doc_chunks = text_splitter.split_documents(data)

Select the embedding model

In [7]:
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

Vector embeddings & Vector database store

In [8]:
vector_db = Chroma.from_documents(
    documents = doc_chunks,
    embedding = embedding_model, 
    collection_name = "local_rag_diat",
    persist_directory="chromadb")

Save the vector db for future use

In [9]:
vector_db.persist()

/Users/raghava/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


Select the ollama model

In [18]:
llm = ChatOllama(model="llama3.1:8b")

Access the stored vector database

In [22]:
vectordb_stored = Chroma(
    persist_directory="chromadb", 
    embedding_function=embedding_model)

Chain together

In [23]:
chain = VectorDBQA.from_chain_type(
                                llm=llm, 
                                chain_type="stuff", vectorstore=vectordb_stored)

/Users/raghava/miniconda3/envs/llm/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:290: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


User input 

In [ ]:
prompt = str(input("Enter your query?"))
chain.run(prompt)